<a href="https://colab.research.google.com/github/snwnkang/CS171_Final_vis_Marathon/blob/main/Boston_marathon_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import HTML
import html

In [32]:
year = 2019

In [2]:
boston2019 = pd.read_csv("https://raw.githubusercontent.com/snwnkang/CS171_Final_vis_Marathon/main/data/boston_data/results2019.csv", error_bad_lines=False)

<ipython-input-2-adeea65923ca>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  boston2019 = pd.read_csv("https://raw.githubusercontent.com/snwnkang/CS171_Final_vis_Marathon/main/data/boston_data/results2019.csv", error_bad_lines=False)
Skipping line 26134: expected 29 fields, saw 57



In [3]:
boston2019.head(5)

,place_overall,bib,name,age,gender,city,state,country_residence,contry_citizenship,name_suffix,...,pace,projected_time,official_time,overall,gender_result,division_result,seconds,first_name,last_name,display_name
0,1,2,"Cherono, Lawrence",30,M,Eldoret,NaN,Kenya,Kenya,NaN,...,NaN,NaN,2:07:57,1,1,1,7677.0,NaN,NaN,Lawrence Cherono
1,2,6,"Desisa, Lelisa",29,M,Ambo,NaN,Ethiopi,Ethiopi,NaN,...,NaN,NaN,2:07:59,2,2,2,7679.0,NaN,NaN,Lelisa Desisa
2,3,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,Kenya,Kenya,NaN,...,NaN,NaN,2:08:07,3,3,3,7687.0,NaN,NaN,Kenneth Kipkemoi
3,4,8,"Kandie, Felix",32,M,Iten,NaN,Kenya,Kenya,NaN,...,NaN,NaN,2:08:54,4,4,4,7734.0,NaN,NaN,Felix Kandie
4,5,11,"Kirui, Geoffrey",26,M,Keringet,NaN,Kenya,Kenya,NaN,...,NaN,NaN,2:08:55,5,5,5,7735.0,NaN,NaN,Geoffrey Kirui


In [4]:
columns_to_keep = ['place_overall', 'bib', 'name', 'age', 'gender', 'country_residence', 'official_time', 'seconds']

# Drop all columns except the ones specified
boston2019 = boston2019[columns_to_keep]

# Now boston2019 contains only the columns you want to keep
print(boston2019.head())

   place_overall bib               name age gender country_residence  \
0              1   2  Cherono, Lawrence  30      M             Kenya   
1              2   6     Desisa, Lelisa  29      M           Ethiopi   
2              3   7  Kipkemoi, Kenneth  34      M             Kenya   
3              4   8      Kandie, Felix  32      M             Kenya   
4              5  11    Kirui, Geoffrey  26      M             Kenya   

  official_time  seconds  
0       2:07:57   7677.0  
1       2:07:59   7679.0  
2       2:08:07   7687.0  
3       2:08:54   7734.0  
4       2:08:55   7735.0  


In [5]:
boston2019.head(5)

,place_overall,bib,name,age,gender,country_residence,official_time,seconds
0,1,2,"Cherono, Lawrence",30,M,Kenya,2:07:57,7677.0
1,2,6,"Desisa, Lelisa",29,M,Ethiopi,2:07:59,7679.0
2,3,7,"Kipkemoi, Kenneth",34,M,Kenya,2:08:07,7687.0
3,4,8,"Kandie, Felix",32,M,Kenya,2:08:54,7734.0
4,5,11,"Kirui, Geoffrey",26,M,Kenya,2:08:55,7735.0


In [6]:
def categorize_age(age):
    if 18 <= age <= 19:
        return '18-19'
    elif 20 <= age <= 24:
        return '20-24'
    elif 25 <= age <= 29:
        return '25-29'
    elif 30 <= age <= 34:
        return '30-34'
    elif 35 <= age <= 39:
        return '35-39'
    elif 40 <= age <= 44:
        return '40-44'
    elif 45 <= age <= 49:
        return '45-49'
    elif 50 <= age <= 54:
        return '50-54'
    elif 55 <= age <= 59:
        return '55-59'
    elif 60 <= age <= 64:
        return '60-64'
    elif 65 <= age <= 69:
        return '65-69'
    else:
        return '70+'  # For all ages above 69

In [7]:
unique_ages = boston2019['age'].unique()
unique_ages

array(['30', '29', '34', '32', '26', '28', '27', '24', '35', '31', '36',
       '25', '33', '37', '42', '44', '39', '23', '38', '41', '46', '21',
       '43', '40', '47', '54', '45', '22', '48', '52', '49', '50', '19',
       '20', '53', '55', '51', '18', '57', '59', '60', '62', '63', '68',
       '71', '58', '56', '61', '64', 'M', '70', '65', '66', '67', '69',
       '74', '80', '72', '78', '76', '75', '77', '79', '81', '73', '82',
       '83'], dtype=object)

In [8]:
boston2019 = boston2019[boston2019['age'] != 'M']

In [9]:
boston2019['age'] = boston2019['age'].astype(int)

In [10]:
boston2019['Division'] = boston2019['age'].apply(categorize_age)
boston2019

,place_overall,bib,name,age,gender,country_residence,official_time,seconds,Division
0,1,2,"Cherono, Lawrence",30,M,Kenya,2:07:57,7677.0,30-34
1,2,6,"Desisa, Lelisa",29,M,Ethiopi,2:07:59,7679.0,25-29
2,3,7,"Kipkemoi, Kenneth",34,M,Kenya,2:08:07,7687.0,30-34
3,4,8,"Kandie, Felix",32,M,Kenya,2:08:54,7734.0,30-34
4,5,11,"Kirui, Geoffrey",26,M,Kenya,2:08:55,7735.0,25-29
...,...,...,...,...,...,...,...,...,...
26640,26653,23909,"Mcninch, Brett",42,M,United,7:14:12,26052.0,40-44
26641,26654,23949,"Oconell, Justin",28,M,United,7:16:19,26179.0,25-29
26642,26655,15946,"Young, John",53,M,United,7:17:35,26255.0,50-54
26643,26656,23937,"Smith, Denise",51,F,United,7:25:39,26739.0,50-54


In [11]:
def split_names(name):
    # Split the name at the comma
    parts = name.split(',')
    # The first part is the last name, the second part is the first name
    last_name = parts[0].strip()
    first_name = parts[1].strip() if len(parts) > 1 else ''
    return last_name, first_name

# Apply the function to the 'name' column and create new columns for the split names
boston2019[['Last Name', 'First Name']] = boston2019['name'].apply(
    lambda x: pd.Series(split_names(x), index=['Last Name', 'First Name'])
)

# Display the DataFrame to verify the changes
boston2019[['name', 'Last Name', 'First Name']].head()

,name,Last Name,First Name
0,"Cherono, Lawrence",Cherono,Lawrence
1,"Desisa, Lelisa",Desisa,Lelisa
2,"Kipkemoi, Kenneth",Kipkemoi,Kenneth
3,"Kandie, Felix",Kandie,Felix
4,"Kirui, Geoffrey",Kirui,Geoffrey


In [12]:
boston2019

,place_overall,bib,name,age,gender,country_residence,official_time,seconds,Division,Last Name,First Name
0,1,2,"Cherono, Lawrence",30,M,Kenya,2:07:57,7677.0,30-34,Cherono,Lawrence
1,2,6,"Desisa, Lelisa",29,M,Ethiopi,2:07:59,7679.0,25-29,Desisa,Lelisa
2,3,7,"Kipkemoi, Kenneth",34,M,Kenya,2:08:07,7687.0,30-34,Kipkemoi,Kenneth
3,4,8,"Kandie, Felix",32,M,Kenya,2:08:54,7734.0,30-34,Kandie,Felix
4,5,11,"Kirui, Geoffrey",26,M,Kenya,2:08:55,7735.0,25-29,Kirui,Geoffrey
...,...,...,...,...,...,...,...,...,...,...,...
26640,26653,23909,"Mcninch, Brett",42,M,United,7:14:12,26052.0,40-44,Mcninch,Brett
26641,26654,23949,"Oconell, Justin",28,M,United,7:16:19,26179.0,25-29,Oconell,Justin
26642,26655,15946,"Young, John",53,M,United,7:17:35,26255.0,50-54,Young,John
26643,26656,23937,"Smith, Denise",51,F,United,7:25:39,26739.0,50-54,Smith,Denise


In [14]:
boston2019['Year'] = 2019

In [15]:
rename_columns = {
    'place_overall': 'Overall Place',
    'bib': 'BIB',
    'country_residence': 'Country',
    'gender': 'Gender',
    'official_time': 'Time',
    'seconds': 'Seconds'
}

boston2019.rename(columns=rename_columns, inplace=True)

In [16]:
new_column_order = [
    'Last Name', 'First Name', 'Division', 'Country', 'BIB', 'Overall Place',
    'Time', 'Seconds', 'Year', 'Gender'
]

boston2019 = boston2019[new_column_order]

In [22]:
unique_country = boston2019['Country'].unique()
unique_country

array(['Kenya', 'Ethiopi', 'United ', 'Japan', 'Morocco', 'Mexico',
       'Canada', 'Austral', 'Germany', 'China', 'Dominic', 'Israel',
       'Argenti', 'Ireland', 'Brazil', 'Turkey', 'Austria', 'Peru',
       'France', 'Chile', 'Switzer', 'Denmark', 'Norway', 'Finland',
       'Portuga', 'Sweden', 'Latvia', 'Colombi', 'Bermuda', 'Belgium',
       'Hong Ko', 'Ecuador', 'Italy', 'Costa R', 'Taiwan', 'Estonia',
       'Spain', 'Palesti', 'Venezue', 'Czech R', 'Netherl', 'Russian',
       'India', 'South A', 'Singapo', 'Poland', 'Luxembo', 'Hungary',
       'Guatema', 'Lithuan', 'Ukraine', 'Belarus', 'Sloveni', 'Korea, ',
       'New Zea', 'Puerto ', 'Macao', 'Armenia', 'El Salv', 'Thailan',
       'Nepal', 'Philipp', 'Slovaki', 'Vietnam', 'Faroe I', 'Panama',
       'Andorra', 'Hondura', 'Uruguay', 'Malaysi', 'Iceland', 'Monaco',
       'Indones', 'Trinida', 'Algeria', 'Cyprus', 'Greece', 'Cayman ',
       'Moldova', 'Romania', 'Serbia ', 'Croatia', 'Belize', 'Saudi A',
       'Malta',

In [24]:
country_codes = {
    'Kenya': 'KEN', 'Ethiopi': 'ETH', 'United ': 'USA', 'Japan': 'JPN',
    'Morocco': 'MAR', 'Mexico': 'MEX', 'Canada': 'CAN', 'Austral': 'AUS',
    'Germany': 'DEU', 'China': 'CHN', 'Dominic': 'DMA', 'Israel': 'ISR',
    'Argenti': 'ARG', 'Ireland': 'IRL', 'Brazil': 'BRA', 'Turkey': 'TUR',
    'Austria': 'AUT', 'Peru': 'PER', 'France': 'FRA', 'Chile': 'CHL',
    'Switzer': 'CHE', 'Denmark': 'DNK', 'Norway': 'NOR', 'Finland': 'FIN',
    'Portuga': 'PRT', 'Sweden': 'SWE', 'Latvia': 'LVA', 'Colombi': 'COL',
    'Bermuda': 'BMU', 'Belgium': 'BEL', 'Hong Ko': 'HKG', 'Ecuador': 'ECU',
    'Italy': 'ITA', 'Costa R': 'CRI', 'Taiwan': 'TWN', 'Estonia': 'EST',
    'Spain': 'ESP', 'Palesti': 'PSE', 'Venezue': 'VEN', 'Czech R': 'CZE',
    'Netherl': 'NLD', 'Russian': 'RUS', 'India': 'IND', 'South A': 'ZAF',
    'Singapo': 'SGP', 'Poland': 'POL', 'Luxembo': 'LUX', 'Hungary': 'HUN',
    'Guatema': 'GTM', 'Lithuan': 'LTU', 'Ukraine': 'UKR', 'Belarus': 'BLR',
    'Sloveni': 'SVN', 'Korea, ': 'KOR', 'New Zea': 'NZL', 'Puerto ': 'PRI',
    'Macao': 'MAC', 'Armenia': 'ARM', 'El Salv': 'SLV', 'Thailan': 'THA',
    'Nepal': 'NPL', 'Philipp': 'PHL', 'Slovaki': 'SVK', 'Vietnam': 'VNM',
    'Faroe I': 'FRO', 'Panama': 'PAN', 'Andorra': 'AND', 'Hondura': 'HND',
    'Uruguay': 'URY', 'Malaysi': 'MYS', 'Iceland': 'ISL', 'Monaco': 'MCO',
    'Indones': 'IDN', 'Trinida': 'TTO', 'Algeria': 'DZA', 'Cyprus': 'CYP',
    'Greece': 'GRC', 'Cayman ': 'CYM', 'Moldova': 'MDA', 'Romania': 'ROU',
    'Serbia ': 'SRB', 'Croatia': 'HRV', 'Belize': 'BLZ', 'Saudi A': 'SAU',
    'Malta': 'MLT', 'Kuwait': 'KWT', 'Bolivia': 'BOL', 'Barbado': 'BRB',
    'Martini': 'MTQ', 'Aruba': 'ABW'
}

In [25]:
def replace_country_names(df, country_dict):
    df['Country'] = df['Country'].map(country_dict).fillna(df['Country'])
    return df

In [26]:
boston2019 = replace_country_names(boston2019, country_codes)
boston2019.head()

<ipython-input-25-d5faa6fe5beb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].map(country_dict).fillna(df['Country'])


,Last Name,First Name,Division,Country,BIB,Overall Place,Time,Seconds,Year,Gender
0,Cherono,Lawrence,30-34,KEN,2,1,2:07:57,7677.0,2019,M
1,Desisa,Lelisa,25-29,ETH,6,2,2:07:59,7679.0,2019,M
2,Kipkemoi,Kenneth,30-34,KEN,7,3,2:08:07,7687.0,2019,M
3,Kandie,Felix,30-34,KEN,8,4,2:08:54,7734.0,2019,M
4,Kirui,Geoffrey,25-29,KEN,11,5,2:08:55,7735.0,2019,M


In [27]:
unique_country = boston2019['Country'].unique()
unique_country

array(['KEN', 'ETH', 'USA', 'JPN', 'MAR', 'MEX', 'CAN', 'AUS', 'DEU',
       'CHN', 'DMA', 'ISR', 'ARG', 'IRL', 'BRA', 'TUR', 'AUT', 'PER',
       'FRA', 'CHL', 'CHE', 'DNK', 'NOR', 'FIN', 'PRT', 'SWE', 'LVA',
       'COL', 'BMU', 'BEL', 'HKG', 'ECU', 'ITA', 'CRI', 'TWN', 'EST',
       'ESP', 'PSE', 'VEN', 'CZE', 'NLD', 'RUS', 'IND', 'ZAF', 'SGP',
       'POL', 'LUX', 'HUN', 'GTM', 'LTU', 'UKR', 'BLR', 'SVN', 'KOR',
       'NZL', 'PRI', 'MAC', 'ARM', 'SLV', 'THA', 'NPL', 'PHL', 'SVK',
       'VNM', 'FRO', 'PAN', 'AND', 'HND', 'URY', 'MYS', 'ISL', 'MCO',
       'IDN', 'TTO', 'DZA', 'CYP', 'GRC', 'CYM', 'MDA', 'ROU', 'SRB',
       'HRV', 'BLZ', 'SAU', 'MLT', 'KWT', 'BOL', 'BRB', 'MTQ', 'ABW'],
      dtype=object)

In [30]:
boston2019
#up to here is the full data
boston2019.to_csv("full2019boston.csv")

In [28]:
first_100_M = boston2019[boston2019['Gender'] == 'M'].head(100)

first_100_F = boston2019[boston2019['Gender'] == 'F'].head(100)

In [33]:
first_100_M.to_csv(f"Boston{year}_M.csv")

In [34]:
first_100_F.to_csv(f"Boston{year}_F.csv")